In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import segmentation_models as sm
import gc
from pipeline import Pipeline
import matplotlib.pyplot as plt
from tqdm import tqdm



##### Put this in pipeline.py

In [ ]:

# def predict_and_assemble(self, volume, mask, threshold, model):
#     test_a_ds = self.make_iterated_data_generator(volume, mask)
#     locations = self.list_all_locations(mask)
#     all_pred = np.zeros(mask.shape)
#     all_binary_pred = np.zeros(mask.shape)
#     pred = []

#     for i, patch in tqdm(enumerate(test_a_ds())):
#         # pred[i]: (128, 128, 1)
#         p = model.predict(np.expand_dims(patch, 0))[0]
#         pred.append(p)
#         c = locations[i]
#         l = c[0] - self.patch_halfsize
#         r = c[0] + self.patch_halfsize
#         t = c[1] + self.patch_halfsize
#         b = c[1] - self.patch_halfsize
#         p = p.squeeze()
#         all_pred[l:r, b:t] = p
#         p = p > threshold
#         all_binary_pred[l:r, b:t] = p

#     return all_pred, all_binary_pred, pred


## Load the test set, build the pipeline

In [ ]:
data_dir = "./dataset/"
patch_size = 128  # e.g. 128x128
downsampling = 0.75  # setting this to e.g. 0.5 means images will be loaded as 2x smaller. 1 does nothing.
z_dim = 40   # number of slices in the z direction. max value is 65 - z_start
z_start = 0  # offset of slices in the z direction
batch_size = 16
epochs = 100
steps_per_epoch = 50
val_step = 50

pipeline = Pipeline(data_dir, patch_size, downsampling, z_dim, z_start, batch_size)
volume_a, mask_a, _ = pipeline.load_sample(split="test", index='a')
# volume_b, mask_b, _ = pipeline.load_sample(split="test", index='b')

gc.collect()
print("Loading complete.")

In [ ]:
print(f"volume_a: {volume_a.shape}")
print(f"mask_a: {mask_a.shape}")

In [ ]:
model = sm.Unet(
    'resnet50',
    input_shape=pipeline.get_input_shape(),
    encoder_weights=None,
    classes=1
)
model.load_weights('chkpt/checkpoint')

In [ ]:
threshold = 0.7
all_pred, all_binary_pred, pred = pipeline.predict_and_assemble(volume_a, mask_a, threshold, model)

## Plot the result

In [ ]:
# mask
plt.imshow(mask_a, cmap='gray')

In [ ]:
# binary predictions
plt.imshow(all_binary_pred, cmap='gray')

In [ ]:
# non-binary predictions
plt.imshow(all_pred, cmap='gray')

## Submission file

In [ ]:
# def rle(output):
#     flat_img = np.where(output > 0.4, 1, 0).astype(np.uint8)
#     starts = np.array((flat_img[:-1] == 0) & (flat_img[1:] == 1))
#     ends = np.array((flat_img[:-1] == 1) & (flat_img[1:] == 0))
#     starts_ix = np.where(starts)[0] + 2
#     ends_ix = np.where(ends)[0] + 2
#     lengths = ends_ix - starts_ix
#     return " ".join(map(str, sum(zip(starts_ix, lengths), ())))

In [ ]:
# submission = defaultdict(list)
# for fragment_id, fragment_name in enumerate(test_fragments):
#     submission["Id"].append(fragment_name.name)
#     submission["Predicted"].append(rle(pred_images[fragment_id]))

# pd.DataFrame.from_dict(submission).to_csv("/kaggle/working/submission.csv", index=False)

In [ ]:
# pd.DataFrame.from_dict(submission)